# Lab 03 - Spark

## Information

| Student name       | ID       |
|--------------------|----------|
| Nguyễn Thiên Phúc  | 20127681 |
| Lê Nguyễn Nhật Phú | 20127275 |
| Võ Hiền Hải Thuận  | 20127344 |


## Note

**`Restart Kernel` and `Run all cells` before submitting on Moodle or your work will be counted as 0!**


## Assignment

### Import libraries

In [1]:
import findspark 
findspark.init()

In [2]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import scipy
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/04/24 20:33:02 WARN Utils: Your hostname, DESKTOP-64E0BLG resolves to a loopback address: 127.0.1.1; using 172.22.88.225 instead (on interface eth0)
23/04/24 20:33:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/24 20:33:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Requirement 1

**Problem 1**: Decision Tree

In [3]:
data = spark.read.csv('Absenteeism_at_work.csv', header=True, inferSchema=True)

feature_columns = ['Month of absence', 'Reason for absence', 'Distance from Residence to Work', 'Body mass index']
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

(training_data, testing_data) = data.randomSplit([0.7, 0.3], seed=100)

dt = DecisionTreeClassifier(labelCol='target', featuresCol='features', maxDepth=3)

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[assembler, dt])

model = pipeline.fit(training_data)

# Make predictions.
predictions = model.transform(testing_data)

evaluator = MulticlassClassificationEvaluator(labelCol='target', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

accuracy = evaluator.evaluate(predictions)

y_true = predictions.select(['target']).collect()
y_pred = predictions.select(['prediction']).collect()

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

treeModel = model.stages[2]
# summary only
print(treeModel)
print("Decision Tree - Test Accuracy = %g" % (accuracy))
print("Decision Tree - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Decision Tree Model is :\n" + str(confusionmatrix))

print("The precision score for Decision Tree Model is: " + str(precision))

print("The recall score for Decision Tree Model is: " + str(recall))

IllegalArgumentException: target does not exist. Available: ID, Reason for absence, Month of absence, Day of the week, Seasons, Transportation expense, Distance from Residence to Work, Service time, Age, Work load Average/day , Hit target, Disciplinary_failure, Education, Son, Social drinker, Social smoker, Pet, Weight, Height, Body mass index, Absenteeism_time_in_hours, features

**Problem 2**: Naive Bayes

In [4]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import SparseVector
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Create or get Spark session
spark = SparkSession.builder.getOrCreate()

# Load training data
data = spark.read.load("Absenteeism_at_work.csv", format="csv", header=True, delimiter=",")

# Perform column transformations
data = data.withColumn("MOA", data["Month of absence"] - 0) \
    .withColumn("label", data['Seasons'] - 0) \
    .withColumn("ROA", data["Reason for absence"] - 0) \
    .withColumn("distance", data["Distance from Residence to Work"] - 0) \
    .withColumn("BMI", data["Body mass index"] - 0)

# Assemble features
assembler = VectorAssembler(inputCols=["label", "MOA"], outputCol='features')
data = assembler.transform(data)

# Split the data into train and test
splits = data.randomSplit([0.7, 0.3], 1000)
train = splits[0]
test = splits[1]

# Create Naive Bayes model and set parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# Train the model
model = nb.fit(train)

# Make predictions
predictions = model.transform(test)

# Compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

# Compute confusion matrix, precision, and recall
y_true = data.select("BMI").rdd.flatMap(lambda x: x).collect()
y_pred = data.select("ROA").rdd.flatMap(lambda x: x).collect()

confusionmatrix = confusion_matrix(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='micro')
recall = recall_score(y_true, y_pred, average='micro')

# Print results
print("Naive Bayes - Test set accuracy = " + str(accuracy))
print("The Confusion Matrix for Naive Bayes Model is :\n" + str(confusionmatrix))
print("The precision score for Naive Bayes Model is: " + str(precision))
print("The recall score for Naive Bayes Model is: " + str(recall))

23/04/24 20:33:20 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/04/24 20:33:22 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


Naive Bayes - Test set accuracy = 0.061224489795918366
The Confusion Matrix for Naive Bayes Model is :
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]
The precision score for Naive Bayes Model is: 0.02972972972972973
The recall score for Naive Bayes Model is: 0.02972972972972973


**Problem 3**: Random Forest

In [5]:
# your code goes here
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
data = spark.read.load("Absenteeism_at_work.csv", format="csv", header=True, delimiter=",")

data = data.withColumn("MOA", data["Month of absence"] - 0).withColumn("label", data['Transportation expense'] - 0). \
    withColumn("ROA", data["Reason for absence"] - 0). \
    withColumn("distance", data["Distance from Residence to Work"] - 0). \
    withColumn("BMI", data["Body mass index"] - 0)

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.

assem = VectorAssembler(inputCols=["label", "distance"], outputCol='features')

data = assem.transform(data)

labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

y_true = data.select("BMI").rdd.flatMap(lambda x: x).collect()
y_pred = data.select("ROA").rdd.flatMap(lambda x: x).collect()

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

rfModel = model.stages[2]
print(rfModel)  # summary only
print("Random Forest - Test Accuracy = %g" % (accuracy))
print("Random Forest - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Random Forest Model is :\n" + str(confusionmatrix))

print("The precision score for Random Forest Model is: " + str(precision))

print("The recall score for Random Forest Model is: " + str(recall))


+--------------+-----+------------+
|predictedLabel|label|    features|
+--------------+-----+------------+
|         235.0|235.0|[235.0,11.0]|
|         235.0|235.0|[235.0,11.0]|
|         235.0|235.0|[235.0,11.0]|
|         235.0|235.0|[235.0,11.0]|
|         235.0|235.0|[235.0,11.0]|
+--------------+-----+------------+
only showing top 5 rows

RandomForestClassificationModel: uid=RandomForestClassifier_dce28e2b4f4f, numTrees=10, numClasses=24, numFeatures=2
Random Forest - Test Accuracy = 0.896552
Random Forest - Test Error = 0.103448
The Confusion Matrix for Random Forest Model is :
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]
The precision score for Random Forest Model is: 0.02972972972972973
The recall score for Random Forest Model is: 0.02972972972972973


### Requirement 2

**Dataset**:

- Dataset 1: ...

- Dataset 2: ...

**Implementation**

In [ ]:
# your code goes here